In [6]:
import os
from googlesearch import search
import pandas as pd
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests import get
import json
from bs4 import BeautifulSoup
from selenium_recaptcha import Recaptcha_Solver
from selenium.webdriver.chrome.options import Options
import requests

import time



In [9]:
path = "../../Documents/Ticket Sales.xlsx"
events = pd.read_excel(path, sheet_name ="Events")
socials = pd.read_excel(path, sheet_name ="Socials")

In [ ]:
options =  webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\eric9\\AppData\\Local\\Google\\Chrome\\User Data\Default")

driver = webdriver.Chrome(options=options)

In [11]:
def convert_string_to_number(s):
    s = s.lower().strip()
    # Handle comma-separated numbers
    if ',' in s:
        return int(s.replace(',', ''))
    # Handle suffixes like K, M, B
    elif 'k' in s:
        return int(float(s.replace('k', '')) * 1000)
    elif 'm' in s:
        return int(float(s.replace('m', '')) * 1000000)
    elif 'b' in s:
        return int(float(s.replace('b', '')) * 1000000000)
    else:
        return int(s)
    

## Google Search and Web Scraping with Selenium

In [12]:
# Perform a Google search using Selenium
def get_first_search_result(query):
    driver.get(f"https://www.google.com/search?q={query}")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    first_result = soup.find('div', class_='g')
    if first_result:
        first_link = first_result.find('a')['href']
        return first_link
    
    driver.get(f"https://www.bing.com/search?q={query}")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    first_result = soup.find('li', class_='b_algo')
    if first_result:
        if 'href' in first_result.find('a').attrs:
            first_link = first_result.find('a')['href']
            return first_link
    
    try:
        results = list(search(query, advanced=True, num_results=1))
        if len(results) != 0:
            return results[0].url
    except:
        return None



## Instagram

In [ ]:
class InstagramProfile:
    def __init__(self, artist, username =None, follower_count = 0):
        self.artist = artist
        self.username = username
        self.follower_count = follower_count

    def get_username(self):
        if self.username:
            pass 
        driver.get("https://www.instagram.com/web/search/topsearch/?query=" + self.artist)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        pre_tag = soup.find('pre')
        if pre_tag is not None:
            json_data = json.loads(pre_tag.text)
            users = json_data['users']
            if len(users) != 0:
                self.username = users[0]['user']['username']
        if pd.isna(self.username):
            self.get_username2()
        
    def get_username2(self):
        url = get_first_search_result(("instagram profile " + self.artist))
        if url:
            match = re.search(r'instagram\.com/([^/?]+)', url)
            if match:
                self.username = match.group(1)


    def get_followers(self):
        if self.username is None:
            self.follower_count = 0
        url = "https://www.instagram.com/" +  str(self.username)
        page = get(url, timeout=5)
        soup = BeautifulSoup(page.content, 'html.parser')
        data = soup.find_all('meta', attrs={'property': 'og:description'})
        if len(data) == 0:
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            data = soup.find_all('meta', attrs={'property': 'og:description'}
                                 )
        if len(data) == 0:
            self.follower_count = 0
        else:
            follower_count = data[0].get('content').split()[0]
            self.follower_count = convert_string_to_number(follower_count)    
         
    
    def get_all(self):
        self.get_username()
        self.get_followers()
        return self.username, self.follower_count
    def __str__(self):
        return f"Artist: {self.artist}\nInstagram Username: {self.username}\nFollowers: {self.follower_count}"


## Twitter

In [ ]:
class TwitterProfile:
    def __init__(self, artist, username = None, follower_count = 0):
        self.artist = artist
        self.username = username
        self.follower_count = follower_count
    
    def get_username(self):
        if (self.username):
            pass
        driver.get(f"https://x.com/search?q={self.artist}&f=user")
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Timeline: Search timeline']")))
            search_result = driver.find_element(By.XPATH, "//div[@aria-label='Timeline: Search timeline']")
            link_element = search_result.find_elements(By.XPATH, "//a[@tabindex='-1' and @role='link']")
            href = link_element[1].get_attribute('href')
            self.username = re.search(r'x\.com/([^/?]+)', href).group(1)
        except Exception as e:
            self.get_username2()
            pass
    
    def get_username2(self):
        url = get_first_search_result("twitter account " + self.artist)
        if url:
            match = re.search(r'twitter\.com/([^/?]+)', url)
            if match:
                self.username = match.group(1)


    def get_followers(self):
        if not pd.isna(self.username):
            url = "https://www.x.com/" + str(self.username)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # Wait for the follower element to be present
            followers_xpath = '//a[contains(@href, "/verified_followers") and @dir="ltr" and @role="link"]'
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, followers_xpath)))
                follower_element = driver.find_element(By.XPATH, followers_xpath)
                follower_element_text = follower_element.text.replace(' Followers', '')
                follower_count = convert_string_to_number(follower_element_text)
                self.follower_count = follower_count
            except Exception as e:
                pass
    
    def get_all(self):
        self.get_username()
        self.get_followers()
        if self.follower_count < 1000:
            self.get_username2()
            self.get_followers()
        return self.username, self.follower_count
    
    def __str__(self):
        return f"Artist: {self.artist}\nTwitter Username: {self.username}\nFollowers: {self.follower_count}"


## Spotify

In [23]:
from http import client
import requests
import json


with open("../spotify_credentials.json", "r") as f:
    credentials = json.load(f)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]
auth_url = credentials["auth_url"]
response = requests.post(
    auth_url,
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
)

# Check if the request was successful
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print("Access token:", access_token)
else:
    print("Failed to retrieve access token:", response.status_code, response.text)

headers = {
    "Authorization": f"Bearer {access_token}"
}

    

Access token: BQD2nfHRNnj_CJhGNTlfTqX-pnD6faHbJhOOMG55f8bqHiEeRcJ-k7kgA87G1Vsgl8rGy6UwBhCdFtU0D9Pg-lOGNOuySYdlZFgJ1p5EjG_tjWhXHbc


In [ ]:


class SpotifyProfile:
    def __init__(self, artist, spotifyID = None, genre = None, followers = 0, popularity = 0, listens = 0):
        self.artist = artist
        self.spotifyID = spotifyID
        self.genre = genre
        self.followers = followers
        self.popularity = popularity
        self.listens = listens
        self.url = None

    def get_id(self):
        if self.spotifyID:
            return self.spotifyID
        search_url = f"https://api.spotify.com/v1/search?q=artist:{self.artist}&type=artist&limit=1"
        response = requests.get(search_url, headers=headers)
        if response.status_code == 200:
            search_results = response.json()
            if len(search_results['artists']['items']):
                self.spotifyID = search_results['artists']['items'][0]['id']
        if self.spotifyID is None:
            self.get_id2()

    
    def get_id2(self):
        url = get_first_search_result("spotify  " + self.artist)
        if url:
            match = re.search(r'artist/([^/?]+)', url)
            if match:
                self.spotifyID = match.group(1)

    def get_spot_stats(self):
        search_url = f"https://api.spotify.com/v1/artists/{self.spotifyID}"
        response = requests.get(search_url, headers=headers)
        if response.status_code == 200:
            result = response.json()
            self.spotifyID = result['id']
            if len(result['genres']):
                self.genre = result['genres'][0]
            self.followers = result['followers']['total']
            self.popularity = result['popularity']
            self.url = result['external_urls']['spotify']
            
    def get_listens(self,):
        if self.url is None:
            return None
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        meta_tag = soup.find('meta', attrs={'property': 'og:description'})
        if meta_tag:
            content = meta_tag.get('content')
            match = re.search(r'(\d+(\.\d+)?[MK]?) monthly listeners', content)
            if match:
                self.listens = convert_string_to_number(match.group(1))

    def get_all(self):
        self.get_id()
        self.get_spot_stats()
        self.get_listens()
        if self.listens <90000:
            self.get_id2()
            self.get_spot_stats()
            self.get_listens()
        return self.spotifyID, self.genre, self.followers, self.popularity, self.listens
    
    def __str__(self):
        return f"Artist: {self.artist}\nSpotify ID: {self.spotifyID}\nGenre: {self.genre}\nFollowers: {self.followers}\nPopularity: {self.popularity}\nMonthly Listeners: {self.listens}"


    


In [10]:
# import requests

# search_query = "genre:edm"
# search_type = "artist"
# search_url = f"https://api.spotify.com/v1/search?q={search_query}&type={search_type}"

# response = requests.get(search_url, headers=headers)

# if response.status_code == 200:
#     search_results = response.json()
#     # print(json.dumps(search_results, indent=4))
# else:
#     print("Failed to retrieve search results:", response.status_code, response.text)
# # print(search_results['artists']['items'][0]['followers']['total'])
# # print(search_results['artists']['items'][0]['popularity'])
# # Extract relevant information from search results
# artists = search_results['artists']['items']
# data = []
# for artist in artists:
#     data.append({
#         'name': artist['name'],
#         'followers': artist['followers']['total'],
#         'popularity': artist['popularity'],
#         'genres': ', '.join(artist['genres']),
#         'spotify_url': artist['external_urls']['spotify']
#     })


# # Create a DataFrame from the extracted data
# df_artists = pd.DataFrame(data)
# df_artists = df_artists.sort_values(by='popularity', ascending=False)
# print(df_artists)

## Stubhub


In [ ]:
import pprint

class StubhubProfile:
    def __init__(self, artist, url = None, favourites = 0):
        self.artist = artist
        self.url =  url 
        self.favourites = favourites

    def get_url(self):
        if self.url:
            return self.url
        try:
            pageSource = requests.get("https://www.stubhub.ca/secure/search?q=" + self.artist).content
        except:
            pageSource = driver.get("https://www.stubhub.ca/secure/search?q=" + self.artist).page_source
        soup = BeautifulSoup(pageSource, 'html.parser')
        script_tag = soup.find('script', {'id': 'index-data', 'type': 'application/json'})
        json_data = json.loads(script_tag.string)
        self.url = json_data['topSearchResults'][0]['url']
        return self.url

    def get_favourites(self):
        if self.url is None:
            return None
        pageSource = requests.get("https://www.stubhub.ca/" + self.url).content
        soup = BeautifulSoup(pageSource, 'html.parser')
        script_tag = soup.find('script', {'id': 'index-data', 'type': 'application/json'})
        if script_tag is None:
            return None
        json_data = json.loads(script_tag.string)
        if 'categorySummary' not in json_data:
            return None
        self.favourites = json_data['categorySummary']['aggregateFavorites']
        return self.favourites

    def get_all(self):
        self.get_url()
        self.get_favourites()
        return self.url, self.favourites
    
    def __str__(self):
        return f"Artist: {self.artist}\nURL: {self.url}\nFavourites: {self.favourites}"



    


In [12]:
stubhub = StubhubProfile("illenium")
print(stubhub.get_all())
print(stubhub)

('/illenium-tickets/performer/138422940', 980)
Artist: illenium
URL: /illenium-tickets/performer/138422940
Favourites: 980



## Social Media Profiles Extraction


In [13]:
results = []

In [ ]:
import pprint

# Initialize a list to store the results

# Iterate through each artist in the sales DataFrame
for index, row in events.iterrows():
    if artist in socials['artist'].values:
        continue
    
    artist = row['Artist']
    if any(result['artist'] == artist for result in results):
        print(f"Skipping {artist} as it is already processed.")
        continue

    if artist in socials['Artist'].values:
        print(f"Skipping {artist} as it is already in sales.")
        continue
    print(f"Processing {artist}")
    instagram = InstagramProfile(artist)
    spotify = SpotifyProfile(artist)
    twitter = TwitterProfile(artist)
    stubhub = StubhubProfile(artist)
    instagram.get_all()
    spotify.get_all()
    twitter.get_all()
    stubhub.get_all()
    social = {
        'artist': artist,
        'instagram_username': instagram.username,
        'instagram_followers': instagram.follower_count,
        'spotify_id': spotify.spotifyID,
        'spotify_genre': spotify.genre,
        'spotify_followers': spotify.followers,
        'spotify_popularity': spotify.popularity,
        'spotify_listeners': spotify.listens,
        'twitter_username': twitter.username,
        'twitter_followers': twitter.follower_count,
        'stubhub_url': stubhub.url,
        'stubhub_favourites': stubhub.favourites
    }
    pprint.pprint(social)
   
    results.append(social)
    


In [ ]:
results_df = pd.DataFrame(results)
socials = pd.concat([socials, results_df], ignore_index=True)

In [37]:
with pd.ExcelWriter(path, mode='a', engine="openpyxl", if_sheet_exists="replace") as writer:
    socials.to_excel(writer, sheet_name="Socials", index=False)